In [ ]:
!pip uninstall -y sympy
!pip install sympy==1.12

### **Step 1 - 데이터 로드 + 기본 전처리 + 날짜 파생 + zero-run**

In [ ]:
# ================================
# 📌 Step 1 — 데이터 로드 + 기본 전처리
# ================================
import pandas as pd
import numpy as np
import random
import torch

# -------------------------------
# Seed 고정
# -------------------------------
def set_all_seeds(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    # 연산 일관성을 위해
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_all_seeds(42)

# -------------------------------
# 1) 기본 매출 정리
# -------------------------------
def clean_sales(df):
    df['일매출'] = (
        df['일매출']
        .astype(str)
        .str.replace(",", "")
        .str.replace(" ", "")
        .str.strip()
    )
    df['일매출'] = pd.to_numeric(df['일매출'], errors='coerce').fillna(0)
    return df

# -------------------------------
# 2) 날짜 기반 피처
# -------------------------------
def add_date_features(df):
    df['month'] = df['영업일자'].dt.month
    df['day'] = df['영업일자'].dt.day
    df['weekday'] = df['영업일자'].dt.weekday
    df['is_weekend'] = (df['weekday'] >= 5).astype(int)
    df['weekofyear'] = df['영업일자'].dt.isocalendar().week.astype(int)

    # 영업시간(도메인 지식)
    df['open_hours'] = 11       # 평일 11시간
    df.loc[df['weekday'] == 5, 'open_hours'] = 6
    df.loc[df['weekday'] == 6, 'open_hours'] = 0
    return df

# -------------------------------
# 3) lag/rolling (GRU, Tree, TCN, Transformer 모두에서 사용)
# -------------------------------
def add_lag_rolling(df):
    df['lag1'] = df['일매출'].shift(1)
    df['lag7'] = df['일매출'].shift(7)
    df['lag14'] = df['일매출'].shift(14)
    df['lag28'] = df['일매출'].shift(28)

    df['mean7'] = df['일매출'].rolling(7).mean()
    df['mean28'] = df['일매출'].rolling(28).mean()

    df['std7'] = df['일매출'].rolling(7).std()
    df['std28'] = df['일매출'].rolling(28).std()

    return df

# -------------------------------
# 4) zero-run, zero-fraction 피처
# -------------------------------
def add_zero_features(df):
    # 최근 28일 중 매출 0 비율
    df['zero_frac_28'] = (
        df['일매출'].rolling(28).apply(lambda x: np.mean(x == 0), raw=False)
    )
    # 최근 7일 중 매출 0 비율
    df['zero_frac_7'] = (
        df['일매출'].rolling(7).apply(lambda x: np.mean(x == 0), raw=False)
    )

    # 직전 zero-run-length (연속 0일 수)
    zero_run = 0
    zero_run_list = []
    for v in df['일매출'].values:
        if v == 0:
            zero_run += 1
        else:
            zero_run = 0
        zero_run_list.append(zero_run)
    df['zero_run_tail'] = zero_run_list

    return df

# -------------------------------
# 5) 데이터 불러오기
# -------------------------------
train = pd.read_csv('/content/drive/MyDrive/기계학습/팀플/Data/POS_train_val.csv')
test  = pd.read_csv('/content/drive/MyDrive/기계학습/팀플/Data/POS_test.csv')

train['영업일자'] = pd.to_datetime(train['영업일자'])
test['영업일자']  = pd.to_datetime(test['영업일자'])

train = clean_sales(train)
test = clean_sales(test)

# -------------------------------
# 6) 학사 일정 merge
# -------------------------------
academic = pd.read_csv('/content/drive/MyDrive/기계학습/팀플/Data/학사일정_정리(2325).csv')
academic['영업일자'] = pd.to_datetime(academic['date'])
academic = academic.drop(columns=['date'])

weekday_map = {'mon':0,'tue':1,'wed':2,'thu':3,'fri':4,'sat':5,'sun':6}
academic['acad_weekday'] = academic['weekday'].map(weekday_map)
academic = academic.drop(columns=['weekday'])

academic = academic.rename(columns={
    'weekend':'acad_weekend',
    'holiday':'acad_holiday',
    'semester':'acad_semester',
    'seasonal':'acad_seasonal',
    'exam':'acad_exam',
    'ceremony':'acad_ceremony',
    'dormitory':'acad_dormitory'
})

train = train.merge(academic, on='영업일자', how='left')
test  = test.merge(academic, on='영업일자', how='left')

# -------------------------------
# 7) 날짜 + lag + zero-run 피처 적용
# -------------------------------
train = add_date_features(train)
test  = add_date_features(test)

train = add_lag_rolling(train)
test  = add_lag_rolling(test)

train = add_zero_features(train)
test  = add_zero_features(test)

# -------------------------------
# 8) 학습 가능하도록 dropna
# -------------------------------
train = train.dropna().reset_index(drop=True)

print("📌 Step 1 완료 — train:", train.shape, "/ test:", test.shape)
train.head()


📌 Step 1 완료 — train: (712, 29) / test: (83, 29)


,영업일자,일매출,매출건수,객단가,acad_weekend,acad_holiday,acad_semester,acad_seasonal,acad_exam,acad_ceremony,...,lag7,lag14,lag28,mean7,mean28,std7,std28,zero_frac_28,zero_frac_7,zero_run_tail
0,2023-08-29,201200,53,"3,796",0,0,0,0,0,0,...,147600.0,0.0,0.0,126457.142857,55364.285714,117742.895853,94708.907690,0.678571,0.285714,0
1,2023-08-30,130900,35,"3,740",0,0,0,0,0,0,...,92500.0,0.0,0.0,131942.857143,60039.285714,116787.796982,95104.711646,0.642857,0.285714,0
2,2023-08-31,174900,50,"3,498",0,0,0,0,0,0,...,328900.0,279800.0,0.0,109942.857143,66285.714286,83168.039878,96744.840859,0.607143,0.285714,0
3,2023-09-01,0,0,0,0,0,1,0,0,0,...,89900.0,166700.0,0.0,97100.000000,66285.714286,93124.182323,96744.840859,0.607143,0.428571,1
4,2023-09-02,0,0,0,1,0,1,0,0,0,...,0.0,70900.0,0.0,97100.000000,66285.714286,93124.182323,96744.840859,0.607143,0.428571,2


### **Step 2 - Feature Engineering 고도화**

In [ ]:
# ================================
# 📌 Step 2 — Feature Engineering 고도화
# ================================

from sklearn.preprocessing import LabelEncoder

# -------------------------------
# 1) Winsorizing (이상치 억제) — 99% clipping
# -------------------------------
def winsorize_series(s, q=0.99):
    upper = s.quantile(q)
    return np.where(s > upper, upper, s)

train['일매출_clip'] = winsorize_series(train['일매출'], q=0.99)

# -------------------------------
# 2) 로그 변환 (GRU/LSTM/Transformer 사용할 때 매우 중요)
# -------------------------------
train['일매출_log'] = np.log1p(train['일매출_clip'])
test['일매출_log']  = np.log1p(test['일매출'])

# -------------------------------
# 3) Cyclical Encoding
# -------------------------------
def add_cyclical(df):
    df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
    df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)

    df['dow_sin'] = np.sin(2 * np.pi * df['weekday'] / 7)
    df['dow_cos'] = np.cos(2 * np.pi * df['weekday'] / 7)

    df['doy'] = df['영업일자'].dt.dayofyear
    df['doy_sin'] = np.sin(2 * np.pi * df['doy'] / 365)
    df['doy_cos'] = np.cos(2 * np.pi * df['doy'] / 365)
    return df

train = add_cyclical(train)
test  = add_cyclical(test)

# -------------------------------
# 4) Label Encoding (학사 일정 categorical 변환)
#    - 학사 피처 대부분이 0/1이지만, acad_weekday는 0~6
#    - 필요 시 모든 개별 피처 LabelEncoding 적용
# -------------------------------
le_dict = {}
for col in ['acad_weekday', 'weekday']:
    le = LabelEncoder()
    full = pd.concat([train[col], test[col]], axis=0).fillna(0)
    le.fit(full)

    train[col] = le.transform(train[col].fillna(0))
    test[col] = le.transform(test[col].fillna(0))

    le_dict[col] = le

# -------------------------------
# 5) 최종 Feature Set 생성
# -------------------------------
feature_cols = [
    # 학사 일정
    'acad_weekend','acad_semester','acad_weekday',
    'acad_holiday','acad_seasonal','acad_exam',
    'acad_ceremony','acad_dormitory',

    # 날짜 기반
    'month','day','weekday','is_weekend','weekofyear','open_hours',

    # lag/rolling/statistics
    'lag1','lag7','lag14','lag28',
    'mean7','mean28','std7','std28',

    # zero-features
    'zero_frac_7','zero_frac_28','zero_run_tail',

    # cyclical
    'month_sin','month_cos',
    'dow_sin','dow_cos',
    'doy_sin','doy_cos',
]

# 결측치 채우기
train[feature_cols] = train[feature_cols].fillna(0)
test[feature_cols]  = test[feature_cols].fillna(0)

print("📌 Step 2 완료 — final feature count:", len(feature_cols))
train[feature_cols].head()


📌 Step 2 완료 — final feature count: 31


,acad_weekend,acad_semester,acad_weekday,acad_holiday,acad_seasonal,acad_exam,acad_ceremony,acad_dormitory,month,day,...,std28,zero_frac_7,zero_frac_28,zero_run_tail,month_sin,month_cos,dow_sin,dow_cos,doy_sin,doy_cos
0,0,0,1,0,0,0,0,1,8,29,...,94708.907690,0.285714,0.678571,0,-0.866025,-5.000000e-01,0.781831,0.623490,-0.845249,-0.534373
1,0,0,2,0,0,0,0,1,8,30,...,95104.711646,0.285714,0.642857,0,-0.866025,-5.000000e-01,0.974928,-0.222521,-0.854322,-0.519744
2,0,0,3,0,0,0,0,1,8,31,...,96744.840859,0.285714,0.607143,0,-0.866025,-5.000000e-01,0.433884,-0.900969,-0.863142,-0.504961
3,0,1,4,0,0,0,0,1,9,1,...,96744.840859,0.428571,0.607143,1,-1.000000,-1.836970e-16,-0.433884,-0.900969,-0.871706,-0.490029
4,1,1,5,0,0,0,0,1,9,2,...,96744.840859,0.428571,0.607143,2,-1.000000,-1.836970e-16,-0.974928,-0.222521,-0.880012,-0.474951


### **Step 3 - 모델 학습**

**3-1. GRU**

In [ ]:
# ================================
# 📌 Step 3-1 — GRU Model Training
# ================================

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# -------- Dataset --------
class SeqDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32).unsqueeze(1)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# -------- GRU Model --------
class GRUModel(nn.Module):
    def __init__(self, input_dim, hidden_dim=64, num_layers=2):
        super().__init__()
        self.gru = nn.GRU(
            input_dim, hidden_dim,
            num_layers=num_layers, batch_first=True
        )
        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        out, _ = self.gru(x)
        return self.fc(out[:, -1])

# -------- Prepare Sequences --------
SEQ_LEN = 28

def create_seq(df, feature_cols):
    X, y = [], []
    values = df[feature_cols].values
    target = df['일매출_log'].values
    for i in range(SEQ_LEN, len(df)):
        X.append(values[i-SEQ_LEN:i])
        y.append(target[i])
    return np.array(X), np.array(y)

X_train, y_train = create_seq(train, feature_cols)

dataset = SeqDataset(X_train, y_train)
loader = DataLoader(dataset, batch_size=32, shuffle=True)

# -------- Train GRU --------
gru = GRUModel(input_dim=len(feature_cols), hidden_dim=64, num_layers=2)
optimizer = torch.optim.Adam(gru.parameters(), lr=0.001)
criterion = nn.MSELoss()

EPOCHS = 50
for ep in range(EPOCHS):
    total_loss = 0
    for xb, yb in loader:
        optimizer.zero_grad()
        loss = criterion(gru(xb), yb)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"[GRU Epoch {ep+1}] Loss={total_loss:.4f}")

print("📌 GRU 학습 완료")


[GRU Epoch 1] Loss=1705.6661
[GRU Epoch 2] Loss=955.0397
[GRU Epoch 3] Loss=761.6946
[GRU Epoch 4] Loss=679.8896
[GRU Epoch 5] Loss=653.9631
[GRU Epoch 6] Loss=646.7675
[GRU Epoch 7] Loss=643.6132
[GRU Epoch 8] Loss=649.6743
[GRU Epoch 9] Loss=636.8436
[GRU Epoch 10] Loss=634.4517
[GRU Epoch 11] Loss=628.3987
[GRU Epoch 12] Loss=620.8707
[GRU Epoch 13] Loss=610.3963
[GRU Epoch 14] Loss=594.2251
[GRU Epoch 15] Loss=571.8339
[GRU Epoch 16] Loss=533.5045
[GRU Epoch 17] Loss=460.8632
[GRU Epoch 18] Loss=382.3189
[GRU Epoch 19] Loss=344.0327
[GRU Epoch 20] Loss=317.9876
[GRU Epoch 21] Loss=319.6702
[GRU Epoch 22] Loss=293.4598
[GRU Epoch 23] Loss=276.1353
[GRU Epoch 24] Loss=270.4501
[GRU Epoch 25] Loss=247.8094
[GRU Epoch 26] Loss=240.9477
[GRU Epoch 27] Loss=232.4128
[GRU Epoch 28] Loss=250.8518
[GRU Epoch 29] Loss=235.7824
[GRU Epoch 30] Loss=223.8068
[GRU Epoch 31] Loss=221.8545
[GRU Epoch 32] Loss=224.1373
[GRU Epoch 33] Loss=206.0537
[GRU Epoch 34] Loss=210.7122
[GRU Epoch 35] Loss=20

**3-2. LGBM**

In [ ]:
# ================================
# 📌 Step 3-2 — LightGBM 모델 학습
# ================================

import lightgbm as lgb

lgb_train = lgb.Dataset(train[feature_cols], label=train['일매출_log'])

lgb_params = {
    "objective": "regression",
    "metric": "rmse",
    "learning_rate": 0.03,
    "num_leaves": 31,
    "feature_fraction": 0.8,
    "bagging_fraction": 0.8,
    "bagging_freq": 3,
}

lgb_model = lgb.train(lgb_params, lgb_train, num_boost_round=300)

print("📌 LightGBM 학습 완료")


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000165 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2329
[LightGBM] [Info] Number of data points in the train set: 712, number of used features: 29
[LightGBM] [Info] Start training from score 9.337994
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -

**3-3. XGBoost**

In [ ]:
# ================================
# 📌 Step 3-3 — XGBoost 모델 학습
# ================================

import xgboost as xgb

dtrain = xgb.DMatrix(train[feature_cols], label=train['일매출_log'])

xgb_params = {
    "objective": "reg:squarederror",
    "eta": 0.03,
    "max_depth": 6,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
}

xgb_model = xgb.train(xgb_params, dtrain, num_boost_round=300)

print("📌 XGBoost 학습 완료")


📌 XGBoost 학습 완료


**3-4. Transformer**

In [ ]:
# ================================
# 📌 Step 3-4 — Transformer (Mini)
# ================================

class MiniTransformer(nn.Module):
    def __init__(self, input_dim, d_model=64, num_heads=4, num_layers=2):
        super().__init__()
        self.input_fc = nn.Linear(input_dim, d_model)

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model, nhead=num_heads, batch_first=True
        )
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        self.fc = nn.Linear(d_model, 1)

    def forward(self, x):
        x = self.input_fc(x)
        h = self.encoder(x)
        return self.fc(h[:, -1])

transformer = MiniTransformer(len(feature_cols))

optimizer_t = torch.optim.Adam(transformer.parameters(), lr=0.001)
criterion = nn.MSELoss()

for ep in range(40):
    total_loss = 0
    for xb, yb in loader:
        optimizer_t.zero_grad()
        loss = criterion(transformer(xb), yb)
        loss.backward()
        optimizer_t.step()
        total_loss += loss.item()
    print(f"[Transformer Epoch {ep+1}] Loss={total_loss:.4f}")

print("📌 Transformer 학습 완료")


[Transformer Epoch 1] Loss=1162.1345
[Transformer Epoch 2] Loss=871.5924
[Transformer Epoch 3] Loss=725.6156
[Transformer Epoch 4] Loss=664.9015
[Transformer Epoch 5] Loss=650.9717
[Transformer Epoch 6] Loss=639.5038
[Transformer Epoch 7] Loss=626.4700
[Transformer Epoch 8] Loss=605.9656
[Transformer Epoch 9] Loss=519.4035
[Transformer Epoch 10] Loss=502.1655
[Transformer Epoch 11] Loss=481.4385
[Transformer Epoch 12] Loss=435.8515
[Transformer Epoch 13] Loss=412.5079
[Transformer Epoch 14] Loss=399.0349
[Transformer Epoch 15] Loss=377.3467
[Transformer Epoch 16] Loss=355.1677
[Transformer Epoch 17] Loss=350.9837
[Transformer Epoch 18] Loss=378.0488
[Transformer Epoch 19] Loss=367.7483
[Transformer Epoch 20] Loss=349.8089
[Transformer Epoch 21] Loss=328.0714
[Transformer Epoch 22] Loss=338.2711
[Transformer Epoch 23] Loss=343.0948
[Transformer Epoch 24] Loss=315.6947
[Transformer Epoch 25] Loss=316.2333
[Transformer Epoch 26] Loss=339.6469
[Transformer Epoch 27] Loss=319.6664
[Transfor

### **Step 4 - 앙상블**

**4-1. 개별 모델 예측 함수**

In [ ]:
# ===========================================
# 📌 Step 4-1 — 개별 모델 예측 함수
# ===========================================

# -------- GRU 예측 --------
def predict_gru(model, train_df, test_df, seq_len, feature_cols):
    history = train_df.copy().reset_index(drop=True)
    preds = []

    for i in range(len(test_df)):
        # 최근 seq_len 구간의 raw feature 사용
        seq = history.tail(seq_len)[feature_cols].values
        X = torch.tensor(seq, dtype=torch.float32).unsqueeze(0)

        pred_log = model(X).item()   # log-space 예측
        preds.append(pred_log)

        # history 업데이트
        new_row = test_df.iloc[i].copy()
        new_row["일매출_log"] = pred_log
        history = pd.concat([history, pd.DataFrame([new_row])], ignore_index=True)

    return np.array(preds)


# -------- Transformer 예측 --------
def predict_transformer(model, train_df, test_df, seq_len, feature_cols):
    history = train_df.copy().reset_index(drop=True)
    preds = []

    for i in range(len(test_df)):
        seq = history.tail(seq_len)[feature_cols].values
        X = torch.tensor(seq, dtype=torch.float32).unsqueeze(0)

        pred_log = model(X).item()
        preds.append(pred_log)

        new_row = test_df.iloc[i].copy()
        new_row["일매출_log"] = pred_log
        history = pd.concat([history, pd.DataFrame([new_row])], ignore_index=True)

    return np.array(preds)


# -------- LightGBM --------
def predict_lgb(model, test_df, feature_cols):
    pred_log = model.predict(test_df[feature_cols])
    return pred_log  # log-space 그대로 반환


# -------- XGBoost --------
def predict_xgb(model, test_df, feature_cols):
    dtest = xgb.DMatrix(test_df[feature_cols])
    pred_log = model.predict(dtest)
    return pred_log  # log-space 그대로 반환


**4-2. 모든 모델 예측 실행**

In [ ]:
# ===========================================
# 📌 Step 4-2 — 모든 모델 예측 + 모델별 성능 출력
# ===========================================

def evaluate(actual, pred):
    actual = np.array(actual)
    pred = np.array(pred)

    mae = np.mean(np.abs(actual - pred))
    rmse = np.sqrt(np.mean((actual - pred) ** 2))
    denom = (np.abs(actual) + np.abs(pred)) / 2
    mask = denom != 0
    smape = np.mean(np.abs(actual[mask] - pred[mask]) / denom[mask]) * 100
    return mae, rmse, smape


# -------------------------------------------
# 1) GRU (log → 매출 복원)
# -------------------------------------------
pred_gru = predict_gru(
    gru,
    train,
    test,
    SEQ_LEN,
    feature_cols
)
pred_gru_real = np.expm1(pred_gru)
gru_mae, gru_rmse, gru_smape = evaluate(test["일매출"], pred_gru_real)

# -------------------------------------------
# 2) Transformer
# -------------------------------------------
pred_tr = predict_transformer(
    transformer,
    train,
    test,
    SEQ_LEN,
    feature_cols
)
pred_tr_real = np.expm1(pred_tr)
tr_mae, tr_rmse, tr_smape = evaluate(test["일매출"], pred_tr_real)

# -------------------------------------------
# 3) LightGBM
# -------------------------------------------
pred_lgb = predict_lgb(
    lgb_model,
    test,
    feature_cols
)
pred_lgb_real = np.expm1(pred_lgb)
lgb_mae, lgb_rmse, lgb_smape = evaluate(test["일매출"], pred_lgb_real)

# -------------------------------------------
# 4) XGBoost
# -------------------------------------------
pred_xgb = predict_xgb(
    xgb_model,
    test,
    feature_cols
)
pred_xgb_real = np.expm1(pred_xgb)
xgb_mae, xgb_rmse, xgb_smape = evaluate(test["일매출"], pred_xgb_real)

# -------------------------------------------
# 📌 출력
# -------------------------------------------
print("\n============== 모델별 Shape ==============")
print("GRU        :", pred_gru.shape)
print("Transformer:", pred_tr.shape)
print("LightGBM   :", pred_lgb.shape)
print("XGBoost    :", pred_xgb.shape)

print("\n============== 모델별 성능 ==============")
print(f"[GRU]        MAE={gru_mae:.2f}, RMSE={gru_rmse:.2f}, SMAPE={gru_smape:.2f}%")
print(f"[Transformer] MAE={tr_mae:.2f}, RMSE={tr_rmse:.2f}, SMAPE={tr_smape:.2f}%")
print(f"[LightGBM]   MAE={lgb_mae:.2f}, RMSE={lgb_rmse:.2f}, SMAPE={lgb_smape:.2f}%")
print(f"[XGBoost]    MAE={xgb_mae:.2f}, RMSE={xgb_rmse:.2f}, SMAPE={xgb_smape:.2f}%")



============== 모델별 Shape ==============
GRU        : (83,)
Transformer: (83,)
LightGBM   : (83,)
XGBoost    : (83,)

============== 모델별 성능 ==============
[GRU]        MAE=101480.78, RMSE=133112.90, SMAPE=112.32%
[Transformer] MAE=125161.46, RMSE=161248.95, SMAPE=142.60%
[LightGBM]   MAE=24099.09, RMSE=37613.97, SMAPE=83.20%
[XGBoost]    MAE=25338.23, RMSE=43549.86, SMAPE=84.57%


**4-3. pTree (XGB + LGBM 평균)**

In [ ]:
# ===========================================
# 📌 Step 4-3 — pTree = (XGB + LGBM) / 2   (log-space)
# ===========================================

pTree_log = (pred_lgb + pred_xgb) / 2



**4-4. pNN (GRU + Transformer 가중합)**

In [ ]:
# ===========================================
# 📌 Step 4-4 — pNN = (GRU + Transformer)
# ===========================================

w_gru = 0.7
w_tr  = 0.3

pNN_log = pred_gru * w_gru + pred_tr * w_tr
# pred_gru, pred_tr 도 log-space 출력


**4-5. 최종 블렌딩**

In [ ]:
# ===========================================
# 📌 Step 4-5 — 최종 블렌딩 (log-space)
# ===========================================
# 비율은 dataset에 따라 조절, 기본 0.5 / 0.5

final_log = 0.8 * pTree_log + 0.2 * pNN_log


# -------- log → real 매출 복구 --------
final_pred = np.expm1(final_log)

# 음수 방지
final_pred = np.maximum(final_pred, 0)

y_true = test['일매출'].values
y_pred = final_pred     # 최종 예측값

def MAE(y, yhat):
    return np.mean(np.abs(y - yhat))

def RMSE(y, yhat):
    return np.sqrt(np.mean((y - yhat)**2))

def SMAPE(y, yhat):
    return np.mean(2 * np.abs(y - yhat) / (np.abs(y) + np.abs(yhat) + 1e-9)) * 100

mae = MAE(y_true, y_pred)
rmse = RMSE(y_true, y_pred)
smape = SMAPE(y_true, y_pred)

print("\n============== 최종 앙상블 성능 ==============")
print(f"[Final Ensemble] MAE={mae:,.2f}, RMSE={rmse:,.2f}, SMAPE={smape:.2f}%")
print("==============================================\n")



============== 최종 앙상블 성능 ==============
[Final Ensemble] MAE=32,789.64, RMSE=52,383.47, SMAPE=87.03%



In [ ]:
# ===========================================
# 📌 최종 블렌딩 = pTree
# ===========================================

final_log = pTree_log
final_pred = np.expm1(final_log)
final_pred = np.maximum(final_pred, 0)

# -------- log → real 매출 복구 --------
final_pred = np.expm1(final_log)

# 음수 방지
final_pred = np.maximum(final_pred, 0)

y_true = test['일매출'].values
y_pred = final_pred     # 최종 예측값

def MAE(y, yhat):
    return np.mean(np.abs(y - yhat))

def RMSE(y, yhat):
    return np.sqrt(np.mean((y - yhat)**2))

def SMAPE(y, yhat):
    return np.mean(2 * np.abs(y - yhat) / (np.abs(y) + np.abs(yhat) + 1e-9)) * 100

mae = MAE(y_true, y_pred)
rmse = RMSE(y_true, y_pred)
smape = SMAPE(y_true, y_pred)


# 평가
y_true = test['일매출'].values

print("\n============== 최종 앙상블 성능 ==============")
print(f"[pTree Only] MAE={MAE(y_true, final_pred):,.2f}, "
      f"RMSE={RMSE(y_true, final_pred):,.2f}, "
      f"SMAPE={SMAPE(y_true, final_pred):.2f}%")
print("==============================================\n")





============== 최종 앙상블 성능 ==============
[pTree Only] MAE=22,131.77, RMSE=37,437.61, SMAPE=82.47%



In [ ]:
# ===========================================
# 📌 Step 4-6 — CSV 저장
# ===========================================

output = test[['영업일자']].copy()
output["예측매출"] = final_pred

output.to_csv("final_ensemble_prediction.csv", index=False)

print("📌 저장 완료 → final_ensemble_prediction.csv")
print(output.head())


📌 저장 완료 → final_ensemble_prediction.csv
        영업일자           예측매출
0 2025-08-10       0.036980
1 2025-08-11       1.716070
2 2025-08-12  114399.206276
3 2025-08-13  113715.275533
4 2025-08-14       2.211638
